# Лабораторная работа №3
## Выполнил студентка группы БВТ2001 Павлова Лада

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [1]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

from tabulate import tabulate
import time
from queue import PriorityQueue

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [2]:
def prefix(template):#программка делает массив количества повторений, т.е.
    array = [0]*len(template)#из лилила получится массив
    countJ = 0#0 0 1 2 3 0
    countI = 1# первые 2 нуля показывают, что повторений символов никаких не было
        #первая единица отвечает за вторую л (лиЛила), которая совпала с первой
        #двойка говорит о том, что уже было два повторения, т.е. ЛИлила, совпало с лиЛИла
        #тройка - ЛИЛила с лиЛИЛа
        #последний ноль - буква А ранеее не встречалась, поэтому повторений не обнаруженно    
    while countI < len(template): 
        if template[countJ] == template[countI]:#если буквы совпали, то увеличиваем все счетчики и цифру в массивве на один
            array[countI] = countJ+1 
            countI += 1
            countJ += 1
        else:
            if countJ == 0:#иначе, если это первый элемент, то оставляем цифру в массиве 0, а i увеличиваем на 1
                array[countI] = 0
                countI += 1
            else:
                countJ = array[countJ-1]#если все остальное не подходит под условие, то приравниваем к элементу массива значение предыдущего элемента
    return array
test = 'лилила'
prefix (test)


[0, 0, 1, 2, 3, 0]

In [3]:
def kmp(original, template):
    lenTemplate = len(template)#берем длинны обеих строк
    lenOriginal = len(original)#искомой и той, в которой ищем
    prefixArray=prefix(template)#вызываем префикс, чтобы создать массив совпадений
    countI = 0
    countJ = 0
    while countI < lenOriginal:#пока i меньше длинны изначальной
        if original[countI] == template[countJ]:#если символы совпдаают
            countI += 1#увеличиваем на 1 счетчики
            countJ += 1
            if countJ == lenTemplate:#если дошли до конца искомой строки, то выводим, что нашли строку
                print(f"example found, index {countI-lenTemplate}")
                break
        else:
            if countJ > 0:#если j Больше нуля, то приравниваем ему предыдущее значение из массива совпадений
                countJ = prefixArray[countJ-1]
            else:#иначе просто увеличиваем i
                countI += 1

    if countI == lenOriginal and countJ != lenTemplate:#пройдясь по всему циклу и не найдя строчку, т.е. i= длинне строки а, j!= длине строке m, то выводим что не нашли
        print("example not found")

#### Упрощенный алгоритм Бойера-Мура

In [4]:
# Этап 1: формирование таблицы смещений
def go_symb(template):
    UniqSymbol = set()  # уникальные символы в образе
    Counts = len(template) # число символов в образе
    dictionary = {}     # словарь смещений

    for i in range(Counts-2, -1, -1): # итерации с предпоследнего символа
        if template[i] not in UniqSymbol:        # если символ еще не добавлен в таблицу
            dictionary[template[i]] = Counts-i-1
            UniqSymbol.add(t[i])

    if template[Counts-1] not in UniqSymbol:     # отдельно формируем последний символ
        dictionary[template[Counts-1]] = Counts

    dictionary['*'] = Counts              # смещения для прочих символов

    return dictionary

# Этап 2: поиск образа в строке
def bm(str,template):
    lenOriginal = len(str)
    lenTemplate = len(template)
    dictionary = go_symb(template)
    if lenOriginal >= lenTemplate:
        i = lenTemplate-1       # счетчик проверяемого символа в строке

        while(i < lenOriginal):
            k = 0
            j = 0
            flBreak = False
            for j in range(lenTemplate-1, -1, -1):
                if str[i-k] != template[j]:
                    if j == lenTemplate-1:
                        off = dictionary[str[i]] if dictionary.get(str[i], False) else dictionary['*']  # смещение, если не равен последний символ образа
                    else:
                        off = dictionary[template[j]]   # смещение, если не равен не последний символ образа

                    i += off    # смещение счетчика строки
                    flBreak = True  # если несовпадение символа, то flBreak = True
                    break

                k += 1          # смещение для сравниваемого символа в строке

            if not flBreak:          # если дошли до начала образа, значит, все его символы совпали
                print(f"exmaple found, index {i-k+1}")
                break
        else:
            print("example not found")
    else:
        print("example not found")

In [5]:
a = 'here is a simple example'
t = 'example'
kmp(a,t)
bm(a,t)

example found, index 17
exmaple found, index 17


In [6]:
a = 'qwertyupopasdnjlkxzqwerewqitpowqqwertyqoitqjoi1111qwertyupopasdnjlkxzqwerewqitpowqqwertyqoitqjoi'
t = '1111'
all_search = dict(
    КНП = kmp,
    БМ = bm,
)
winner = ''
min_time = -1
for name,callable in all_search.items():
    start = time.perf_counter_ns()
    callable(a,t)
    finish = time.perf_counter_ns()
    timeout = finish-start
    if(min_time == -1 or min_time > timeout):
        min_time = timeout
        winner = name
    print(f"{name} is {timeout}")

print(f"{winner} быстрее чем другие!")

example found, index 46
КНП is 308100
exmaple found, index 46
БМ is 62900
БМ быстрее чем другие!


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [7]:
N = 4

# Функция, определяющая движение пятнашек
def moves(position):
    blank = position.index(0)
    # Возвращаются значения из кортежа с целой частью и остатком
    i, j = divmod(blank, N)
    # Задаем массив из смещений
    offsets = []
    # Движение вниз
    if i > 0: offsets.append(-N)     
    # Движение вверх
    if i < N - 1: offsets.append(N)  
    # Движение вправо
    if j > 0: offsets.append(-1)
    # Движение влево
    if j < N - 1: offsets.append(1)  
    for offset in offsets:
        # Заполянем пустую ячейку элементом, который мы двигаем
        swap = blank + offset
        # Проверяем на то, является ли заменяемая ячейка пустой,
        # и в случае, если у нас есть следующий элемент (не конец),
        # то выводим ее индекс, а иначе индекс последнего элемента
        yield tuple(position[swap] if x==blank else position[blank] if x==swap else position[x] for x in range(N*N))

# Функция, определяющая есть ли у пятнашек конечное решение
def parity(permutation):
    # Создаем набор из уникальных данных с пройденными ячейками
    # и создаем кол-во циклов прохождения по массиву ячеек
    seen, cycles = set(), 0
    for i in permutation:
        # Проверяем на наличие текущей ячейки в нашем наборе при перестановке
        # и добавляем его в набор при его отсутствии в нем
        if i not in seen:
            # Наращиваем счетчик циклов при отсутствии значений в наборе
            cycles += 1
            while i not in seen:
                seen.add(i)
                i = permutation[i]
    return (cycles + len(permutation)) % 2

# Класс, определяющий параметры заданного набора
class Position:
    # Конструктор, принимающий позицию и начальную дистанцию
    def __init__(self, position, start_distance):
        self.position = position
        self.start_distance = start_distance

    # Функция, срабатывающая при сравнении (<) объекта с другим объектом
    def __lt__(self, other):
        return self.start_distance < other.start_distance

    # Функция, срабатывающая при использовании объекта как строки
    # и создающая пятнашки в виде квадратной матрицы
    def __str__(self):
        return '\n'.join((N*'{:3}').format(*[i%(N*N) for i in self.position[i:]]) 
for i in range(0, N*N, N))

SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
start = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0]

# Проверяем, можно ли в данной расстановке найти решение. Если нет, то выводим предупреждающее сообщение
if parity(start) == 0:
    print('Can\'t solved')
else:
    # Иначе заполняем наш кортеж значниями из данного набора
    start = tuple(start)

    p = Position(start, 0)

    print(p)
    print()

    # Кладем в очередь с приоритетом первоначальную позицию
    candidates = PriorityQueue()
    candidates.put(p)

    # Кортеж посещенных нами позиций
    visited = set([p])
    # Словарь из позиции в качестве ключа, значение которого в дальнейшем
    # будем заполнять теми ячейками, которые стоит переместить для получения решения
    came_from = {p.position: None}

    while p.position != SOLVED:
        # Извлекаем из очереди позицию с наименьшим приоритетом
        p = candidates.get()
        # Кладем в очередь все соседние позиции. Повторяем, пока не вытащим конечную позицию из очереди
        for k in moves(p.position):
            if k not in visited:
                candidates.put(Position(k, p.start_distance + 1))
                came_from[k] = p
                visited.add(k)

    # Последовательное решение пятнашек (путь)
    path = []
    # Сохраняем конечную позицию
    prev = p
    # Идем в обратном порядке и запоминаем очередность хода в path
    while p.position != start:
        # Запоминаем, откуда ход
        p = came_from[p.position]
        number = p.position[prev.position.index(0)]
        path.append(number)
        prev = p
    path.reverse()

    print(path)

  1  2  3  4
  5  6  7  8
 13  9 11 12
 10 14 15  0

[15, 14, 10, 13, 9, 10, 14, 15]
